The code for creating the baseline comes from [this Jupyter notebook](https://github.com/marshmellow77/text-summarisation-project/blob/main/1_baseline_model.ipynb).

In [ ]:
#!pip install "datasets==1.6.2" --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.8/221.8 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 5.4 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0


In [ ]:
#!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=8f6bb3b55da7fff8883f4921e271ac647ad0c8cca7d546c01a304bc5206b54bc
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score


In [ ]:
import re
import tensorflow_datasets as tfds
import pandas as pd
from sklearn.model_selection import train_test_split
import datasets

In [ ]:
data = tfds.load("cnn_dailymail")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cnn_dailymail/3.4.0.incompleteNMF7HE/cnn_dailymail-validation.tfrecord*...…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cnn_dailymail/3.4.0.incompleteNMF7HE/cnn_dailymail-test.tfrecord*...:   0%…

Dataset cnn_dailymail downloaded and prepared to /root/tensorflow_datasets/cnn_dailymail/3.4.0. Subsequent calls will reuse this data.


In [ ]:
from datasets import load_metric
metric = load_metric("rouge")

In [ ]:
train_data, test_data = data["train"], data["test"]
train_df, test_df = tfds.as_dataframe(train_data), tfds.as_dataframe(test_data)

In [ ]:
val_df = tfds.as_dataframe(data["validation"])

Create the baselines using the first, second, and third sentences for each row in the dataframe.

In [ ]:
display(train_df.head(5))
display(test_df.head(5))

,article,highlights,id,publisher
0,"b""By. Associated Press. PUBLISHED:. 14:11 EST,...","b'Bishop John Folda, of North Dakota, is takin...",b'160e9e1c25c81788df73b06b0ce955897f02ec42',b'dm'
1,b'(CNN) -- Ralph Mata was an internal affairs ...,b'Criminal complaint: Cop used his role to hel...,b'3b07f5102c69e3e609d73b2ccb0dc5549d4fbaf6',b'cnn'
2,"b""A drunk driver who killed a young woman in a...","b""Craig Eccleston-Todd, 27, had drunk at least...",b'2fd252e716c21fd523becd24774bdd6220e1048d',b'dm'
3,"b""(CNN) -- With a breezy sweep of his pen Pres...","b""Nina dos Santos says Europe must be ready to...",b'6e3c654d92c1f34c2355a2a392fb5ef618fddaf8',b'cnn'
4,"b""Fleetwood are the only team still to have a ...",b'Fleetwood top of League One after 2-0 win at...,b'df5e0d63e3cd78096cdbf073232a3aa5f6e3744a',b'dm'


,article,highlights,id,publisher
0,"b""Ever noticed how plane seats appear to be ge...",b'Experts question if packed out planes are p...,b'92c514c913c0bdfe25341af9fd72b29db544099b',b'dm'
1,"b""A drunk teenage boy had to be rescued by sec...","b""Drunk teenage boy climbed into lion enclosur...",b'2003841c7dc0e7c5b1a248f9cd536d727f27a45a',b'dm'
2,"b""Dougie Freedman is on the verge of agreeing ...","b""Nottingham Forest are close to extending Dou...",b'91b7d2311527f5c2b63a65ca98d21d9c92485149',b'dm'
3,"b""Liverpool target Neto is also wanted by PSG ...",b'Fiorentina goalkeeper Neto has been linked w...,b'caabf9cbdf96eb1410295a673e953d304391bfbb',b'dm'
4,"b""Bruce Jenner will break his silence in a two...","b""Tell-all interview with the reality TV star,...",b'3da746a7d9afcaa659088c8366ef6347fe6b53ea',b'dm'


In [ ]:
df = pd.concat([train_df, val_df, test_df], ignore_index = True)
df = df.astype(str)
df["article"] = df["article"].str[1:-1]
df["highlights"] = df["highlights"].str[1:-1]
df["id"] = df["id"].str[1:-1]
df["publisher"] = df["publisher"].str[1:-1]

In [ ]:
ref_summaries = df["highlights"]
display(ref_summaries.head(5))

0    'Bishop John Folda, of North Dakota, is taking...
1    'Criminal complaint: Cop used his role to help...
2    "Craig Eccleston-Todd, 27, had drunk at least ...
3    "Nina dos Santos says Europe must be ready to ...
4    'Fleetwood top of League One after 2-0 win at ...
Name: highlights, dtype: object

In [ ]:
def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

In [40]:
for i in range(3):
  candidate_summaries = list(df["article"].apply(lambda x: " ".join(re.split(r"(?<=[.:;])\s", x)[:i+1])))
  print(f"Scores: {calc_rouge_scores(candidate_summaries, ref_summaries)}")

Scores: {'rouge1': 16.7, 'rouge2': 5.6, 'rougeL': 11.8, 'rougeLsum': 11.8}
Scores: {'rouge1': 22.8, 'rouge2': 8.7, 'rougeL': 14.8, 'rougeLsum': 14.8}
Scores: {'rouge1': 26.2, 'rouge2': 10.4, 'rougeL': 17.0, 'rougeLsum': 17.0}
